# Calculating Noise Parameters from Mulitiple Connected Noisy Networks

This is a simple example showing how a MultiNetworkSystem object correctly calculates noise parameters

In [ ]:
import skrf as rf
import numpy as npy
from skrf.constants import *

frequency = rf.Frequency(start=1000, stop=2000, npoints=201, unit='MHz')
ntwkAttn1 = rf.components.Attenuator(5, frequency=frequency)
ntwkAttn2 = rf.components.Attenuator(7, frequency=frequency)
ntwkC = rf.cascade_2port(ntwkAttn1, ntwkAttn2)

sys = rf.MultiNetworkSystem()

# add all the networks to the system
sys.add(ntwkAttn1, 'attn1')
sys.add(ntwkAttn2, 'attn2')

# make all the connections between the elements
sys.connect('attn1', 2, 'attn2', 1)

# set the external port numbers, for this example the reduced network is a two-port network. The following is used to identify port 1 and port 2 of that resulting network.
sys.external_port('attn1', 1, external_port_number=1)
sys.external_port('attn2', 2, external_port_number=2)

# the verify function make sure that all the ports have been asigned and that everything is consistant for the reduction algorithm
(en, e) = sys.verify()
for error in e:
    print(error)
print("Total number of Errors: " + str(en))

# calculate and return the resulting network
ntwk = sys.reduce()
print(ntwk)
print(ntwk.s)
print(ntwk.cs)

A = npy.identity(npy.shape(ntwk.s)[1])
A[0,0] = 0
kb_t_sas = K_BOLTZMANN*290*npy.matmul(ntwk.s,npy.matmul(A,npy.conjugate(ntwk.s.swapaxes(2,1))))

F= 1 + ntwk.cs[:,0,0]/(kb_t_sas[:,0,0])

print(10*npy.log10(npy.abs(F)))

